**BASELINE MODEL**
=============================

---



# ***INICIALITZATION***

In [ ]:
%matplotlib inline
import torch, torchvision
from torch import nn, optim
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torchvision.transforms as transforms
import torch.nn.functional as F

import sys
import time

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("GPU")
else:
    device = torch.device("cpu")
    print("CPU")   

GPU


In [ ]:
# Hyperparameters
input_model_size=100
in_channel = 1
input_size = input_model_size*input_model_size*in_channel
batch_size = 256

dataSet_size = 50000
train_percentage = 0.8
test_percentage = 0.2

m_learning_rate = 1e-2
m_momentum=0.9

num_epochs = 20

train_size = m = int(dataSet_size * train_percentage)
test_size = int(dataSet_size * test_percentage)


In [ ]:
#HIPERPARAMETERS STANDARS
num_classes_h = 12
num_classes_m = 1  

mean_calculated = 0.4888
std_calculated = 0.2603


# ***LOAD DATASET***

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
#Extract data from the zip
from zipfile import ZipFile
root_datset = "/content/drive/MyDrive/TFG/DataSets/Dtaset proto1/Archive/analog_clocks/images.zip"
with ZipFile(root_datset, 'r') as zip:
    zip.extractall()
    print('Done')

Done


In [ ]:
import os
import pandas as pd
from skimage import io
from torch.utils.data import (
    Dataset,
    DataLoader,
)  

class Clocks(Dataset):
    def __init__(self, csv_file, root_dir, transform=True):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, str(index)+'.jpg')
        image = io.imread(img_path)

        y_label1 = int(self.annotations.iloc[index, 0])
        y_label2 = int(self.annotations.iloc[index, 1])/60
        
        if self.transform:
            image = self.transform(image)
            #image = image.view(image.size(1), image.size(2))
            image = nn.functional.interpolate(image, size=input_model_size)
            #print("Image size is", image.size())
            #print(image)
            #sys.exit()
        return (image, y_label1, y_label2)

In [ ]:
# Load Data
dataset = Clocks(
    csv_file="/content/drive/MyDrive/TFG/DataSets/Dtaset proto1/Archive/analog_clocks/label.csv",
    root_dir="/content/images",
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=(mean_calculated, mean_calculated, mean_calculated), std=(std_calculated, std_calculated, std_calculated)),
        transforms. Grayscale ( num_output_channels=1 )
    ]),
)



train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False)

# ***MODEL TIME***

**NN Architecture**

In [ ]:
class basicCNN(nn.Module):
    def __init__(self, num_ftrs, num_classes_h=12, num_classes_m=60):
        super(basicCNN, self).__init__()
        
        #TRY with bias == 0
        self.conv1 = nn.Conv2d(1, 50, kernel_size=(5, 5), stride=2, bias=True)
        self.maxPool1 = nn.MaxPool2d((2, 2), stride=2)
        self.batchNorm1 = nn.BatchNorm2d(50, affine=False)
        
        self.conv2 = nn.Conv2d(50, 100, kernel_size=(3, 3), stride=1, bias=True)
        self.maxPool2 = nn.MaxPool2d((2, 2))
        self.batchNorm2 = nn.BatchNorm2d(100, affine=False)
        
        self.conv3 = nn.Conv2d(100, 150, kernel_size=(3, 3), stride=1, bias=True)
        self.maxPool3 = nn.MaxPool2d((2, 2))
        self.batchNorm3 = nn.BatchNorm2d(150, affine=False)
        
        self.conv4 = nn.Conv2d(150, 200, kernel_size=(3, 3), stride=1, bias=True)
        self.dropout4 = nn.Dropout(0.4)
        
        self.linearh1 = nn.Linear(in_features=200 * 15 * 2, out_features=144)
        self.linearh2 = nn.Linear(in_features=144, out_features=144)
        self.linearh3 = nn.Linear(in_features=144, out_features=num_classes_h)
        
        self.linearm1 = nn.Linear(in_features=200 * 15 * 2, out_features=200, bias=True)
        self.linearm2 = nn.Linear(in_features=200, out_features=200, bias=True)
        self.linearm3 = nn.Linear(in_features=200, out_features=100, bias=True)
        self.linearm4 = nn.Linear(in_features=100, out_features=num_classes_m)
        
        self.activationConvsLayer = nn.ReLU(inplace=True)
        self.activationFullyLayerHour = nn.ReLU(inplace=True)
        self.activationFullyLayerMinute = nn.ReLU(inplace=True)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.activationConvsLayer(x)
        x = self.maxPool1(x)
        x = self.batchNorm1(x)
        
        x = self.conv2(x)
        x = self.activationConvsLayer(x)
        x = self.maxPool2(x)
        x = self.batchNorm2(x)
        
        x = self.conv3(x)
        x = self.activationConvsLayer(x)
        x = self.maxPool3(x)
        x = self.batchNorm3(x)

        x = self.conv4(x)
        x = self.activationConvsLayer(x)
        x = self.dropout4(x)
        #print(x.size())

        x = x.view(x.size(0), -1)
        
        hour = self.linearh1(x)
        hour = self.activationFullyLayerHour(hour)
        
        hour = self.linearh2(hour)
        hour = self.activationFullyLayerHour(hour)
        
        hour = self.linearh3(hour)


        minute = self.linearm1(x)
        minute = self.activationFullyLayerMinute(minute)
        
        minute = self.linearm2(minute)
        minute = self.activationFullyLayerMinute(minute)
        
        minute = self.linearm3(minute)
        minute = self.activationFullyLayerMinute(minute)
        
        minute = self.linearm4(minute)
        minute = minute.view(-1)

        return hour, minute

**CHECK ACURACY**

In [ ]:
def check_accuracy(loader, model, type_of_data):
    
    print("Metrics of ", type_of_data, " :")

    model.eval()

    num_correct_h = 0
    num_samples_h = 0

    num_correct_m = 0
    num_samples_m = 0

    running_mae = 0
    runnning_mse = 0

    
    with torch.no_grad():
        for data, targeth, targetm in loader:
            data = data.to(device=device)
            targeth = targeth.to(device=device)
            targetm = targetm.to(device=device)

            scores_h, scores_m = model(data)
            _, predictions_h = scores_h.max(1)
            
            num_correct_h += (predictions_h == targeth).sum()
            num_samples_h += predictions_h.size(0)
            
            error = torch.abs(scores_m - targetm).sum().data
            running_mae += error
            
            squared_error = ((scores_m - targetm)*(scores_m - targetm)).sum().data
            runnning_mse += squared_error

            num_samples_m += scores_m.size(0)

        print(
            f"Got {num_correct_h} / {num_samples_h} for HOURS with accuracy {float(num_correct_h)/float(num_samples_h)*100:.2f}%"          
        )
        print(
            f"For MINUTES we have mae {running_mae/num_samples_m*100:.2f}%"
        )
        print(
            f"For MINUTES we have mse {runnning_mse/num_samples_m*100:.2f}%"
        )

    model.train()

**LET'S TRAIN OUR NETWORKY**

In [ ]:
#Charge model:
# Initialize the model for this run
model = basicCNN(input_size, num_classes_h=num_classes_h, num_classes_m=num_classes_m )
model=model.to(device=device)

# Optimize only the classifier
optimizer = optim.SGD(model.parameters(), lr=m_learning_rate, momentum=m_momentum)
#optimizer = optim.Adam(model.parameters(), lr=m_learning_rate)

#Criterion
criterion1 = nn.CrossEntropyLoss()
criterion2 = nn.MSELoss()


#data, targeth, targetm = next(iter(train_loader))


for epoch in range(num_epochs):
    tic = time.time()
    toolbar_width = 40

    # setup toolbar
    sys.stdout.write("[%s]" % (" " * toolbar_width))
    sys.stdout.flush()
    sys.stdout.write("\b" * (toolbar_width+1)) # return to start of line, after '['
    for batch_idx, (data, targeth, targetm) in enumerate(train_loader):
        
        # Get data to cuda if possible
        data = data.to(device=device)
        targeth = targeth.to(device=device)
        targetm = targetm.to(device=device)

        # forward
        out1, out2 = model(data)
        # print(out2.size())
        loss1 = criterion1(out1, targeth)
        loss2 = criterion2(out2, targetm.float())
        loss = loss1 + loss2


        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()
        
        # update the bar
        if batch_idx%20==0:
            sys.stdout.write("-")
            sys.stdout.flush()

    sys.stdout.write("] - 100%\n") # this ends the progress bar

    toc = time.time()

    print("Esta epoch ha tardado: " + str((toc-tic)/60) + " minutos")
check_accuracy(train_loader, model, "train")
check_accuracy(test_loader, model, "test")

[                                        ]

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


--------] - 100%
Esta epoch ha tardado: 2.0156686067581178 minutos
[                                        ]--------] - 100%
Esta epoch ha tardado: 2.0173168818155927 minutos
[                                        ]--------] - 100%
Esta epoch ha tardado: 2.029897566636403 minutos
[                                        ]--------] - 100%
Esta epoch ha tardado: 2.003063416481018 minutos
[                                        ]--------] - 100%
Esta epoch ha tardado: 2.016110157966614 minutos
[                                        ]--------] - 100%
Esta epoch ha tardado: 2.0093503912289936 minutos
[                                        ]--------] - 100%
Esta epoch ha tardado: 2.01650839249293 minutos
[                                     

In [ ]:
print("FINISH")

FINISH
